In [1]:
import numpy as np
import sys, os
import struct
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split

In [2]:
def load_data():
    fp_image = open('train-images-idx3-ubyte', 'rb')
    fp_label = open('train-labels-idx1-ubyte', 'rb')


    dataSet = []
    labelSet = []
    #img = np.zeros((28,28))

    s = fp_image.read(16)
    l = fp_label.read(8)

    dataCount = 0
    while True :
        s = fp_image.read(784)
        l = fp_label.read(1)
    
        if not s:
            break
        if not l:
            break
    
        index = int(l[0])
        labelSet.append(index)
    
        arr = struct.unpack(len(s)*'B', s)
        img = np.reshape( arr, (784))
    
    
        dataSet.append(img)
        dataCount = dataCount + 1
    
    train_data=np.array(dataSet)
    train_label=np.array(labelSet)

    train_label=np.array(labelSet)
    five_index=np.where(train_label==5)
    one_index=np.where(train_label==1)
    eight_index=np.where(train_label==8)

    new_train=[]
    new_label=[]

    for i in five_index[0]:
        new_train.append(train_data[i])
        new_label.append(labelSet[i])
    
    for i in one_index[0]:
        new_train.append(train_data[i])
        new_label.append(labelSet[i])
    
    
    for i in eight_index[0]:
        new_train.append(train_data[i])
        new_label.append(labelSet[i])
        
        
    
    
    x_data=np.array(new_train)
    x_data=x_data/255.
    y_data=np.array(new_label)
    x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,random_state=50,test_size=0.4)
    
    #return x_data,y_data
    return x_train,x_test,y_train,y_test
    
  

In [3]:
class mlp:
    
    def __init__(self,n_output,n_features,n_hidden=30,epochs=500,learning_rate=0.001,shuffle=True,minibatches=1):
        self.n_output=n_output
        self.n_features=n_features
        self.n_hidden=n_hidden
        
        self.w1,self.w2=self.init_weight()
        
        self.epochs=epochs
        self.learning_rate=learning_rate
        
        self.minibatches=minibatches
        
        
    
    def init_weight(self):
        #초기노드 
        # bias 까지 추가한 weight 초기화 -1~1사이의 분포를 따르도록
        # w1 shape (hidden node num , feature+1)
        # W1*x를 해야 하기 때문에 x는 (feature+1 , 총 샘플 수 혹은 미니배치 수)
        # w2 (c 클래스수,  hidden_node_num+1) 
        # W2*z 를 해야하는데 z에서 bias노드 추가를 해야하기 때문에 (hidden_node+1, 샘플수 )
        w1 = np.random.uniform(-1.0,1.0 , size=(self.n_hidden,self.n_features+1))
        w2 = np.random.uniform(-1.0,1.0 , size=(self.n_output, self.n_hidden+1))
          
        
        return w1,w2
    
    
    def one_hot(self,y,n_class):
        #(class수, 샘플 수)
        onehot=np.zeros((n_class, y.shape[0]))
        
        #y: [5,1,8 .......]
        '''for idx,val in enumerate(y):
            onehot[val,idx] = 1.0
            
        
        return onehot'''
        
        
        for idx,val in enumerate(y):
            if val==1:
                onehot[0][idx]=1.0
            
            elif val==5:
                onehot[1][idx]=1.0
                
            
            elif val==8:
                onehot[2][idx]=1.0
                
                
        return onehot        
            
        
        
    
    
    

    def relu(self,x):
        return x*(x>0)
    
    def sigmoid(self,x):
        return 1 / (1 +np.exp(-x))
    
    
    def gradient_of_sig(self,x):
        c=self.sigmoid(x)
        
        return c * (1 - c)
    
   
    
    
    def add_bias(self,x,how='column'):
        #x: (n,784)
        # 첫열에 bias 열 추가 
        
        
        if how=='column':
            bias=np.ones((x.shape[0],1))
            bias_added=np.hstack((bias,x))
            
        elif how=='row':
            bias=np.ones((1,x.shape[1]))
            bias_added=np.vstack((bias,x))
            
            
        return bias_added    
            
        
    
        
        
            
    
    def forward(self,x,w1,w2):
        # x: (n,784)
        # x_bias (n,785)
        # w1 (hidden node 수 , 785)
        # w2(10,31) => (class num , hidden+1)
        # zsum: (hidden node 수 , n)
        # z : (hidden node 수 , n)
        # z_bias: (hidden node+1, n)
        # osum : ( class num , n)
        # bias_x input에 bias 더함, zsum은 hidden node input, z: hidden layer output, osum: outputlayer input
        # o: output (class num, n)
        # y_enc: (class num , n)
        
        #input -> hidden layer 1
        bias_x=self.add_bias(x,'column')
        zsum=w1.dot(bias_x.T)
        z=self.sigmoid(zsum)
        #z=self.relu(zsum)
        
        #hidden -> output layer 
        z_bias=self.add_bias(z,'row')
        osum=w2.dot(z_bias)
        
        #final output 
        output=self.sigmoid(osum)
        
        
        return bias_x , zsum , z , osum , output
        
        
        
        
    
    def backpropagation(self,bias_x,z,output,zsum,y_enc,w1,w2, osum):
        # x: (n,784)
        # x_bias (n,785)
        # w1 (hidden node 수 , 785)
        # w2(10,31) => (class num , hidden+1)
        # zsum: (hidden node 수 , n)
        # z : (hidden node 수 , n)
        # z_bias: (hidden node+1, n)
        # osum : ( class num , n)
        # bias_x input에 bias 더함, zsum은 hidden node input, z: hidden layer output, osum: outputlayer input
        # o: output (class num, n)
        # y_enc: (class num , n)
        
        #output: (class num , 샘플 수)
        #y_enc : (class num, 샘플 수)
        delta_output=(output-y_enc) * self.gradient_of_sig(osum)
        zsum_bias=self.add_bias(zsum,'row')
        
        delta_hidden=w2.T.dot(delta_output) * self.gradient_of_sig(zsum_bias)
        #delta_hidden=w2.T.dot(delta_output) * self.gradient_relu(zsum_bias)
        
        delta_hidden=delta_hidden[1:, :]
        
        gradient_1=delta_hidden.dot(bias_x)
    
        z=self.add_bias(z,'row')
        gradient_2=delta_output.dot(z.T)
        
        return gradient_1,gradient_2
    
    
    
    
    
    def fit(self,x,y,verbose=False):
        x_data,y_data=x.copy(),y.copy()
        y_enc=self.one_hot(y,self.n_output) # one hot encoding
    
        
        for i in range(self.epochs):
            print('epoch',i,' started')
              
            batch_size_t=np.array_split(range(y_data.shape[0]),self.minibatches)
            #array([    0,     1,     2, ..., 18011, 18012, 18013])
            
            for idx in batch_size_t: # 전체데이터를 배치수만큼 나눈다 batch=1 batch_Grdient 
                #전방 계산 
                bias_x, zsum , z , osum , output =self.forward(x[idx],self.w1,self.w2)
                
                
                #역전파 계산 
                grad1 , grad2 = self.backpropagation(bias_x=bias_x, z=z , output=output,zsum=zsum,y_enc=y_enc[:,idx],w1=self.w1,w2=self.w2,osum=osum)
                
                
                # weight 갱신
                
                delta_w1, delta_w2 = self.learning_rate*grad1 ,  self.learning_rate*grad2
                self.w1 -= delta_w1
                self.w2 -= delta_w2
                
                
            print("epoch",i,'  finished')
            
            if verbose==False:
                pass
            
            elif verbose==True:
                print('epoch',i,'weight_1: ', self.w1)
                print('epoch',i,'weight_2: ', self.w2)
                
            
            
                  
            
        return self
    
    
    #def predict(self,x,y):
    #    _, _, _, _, prediction=self.forward(x,self.w1,self.w2)
        
    #    result=np.argmax(prediction,axis=0)
        
    #    is_correct=np.equal(result,y)
        
    #    return np.sum(is_correct.astype(float)) / y.shape[0]
    
    
    
    def prediction(self,x,y):
        _ , _, _, _, prediction= self.forward(x,self.w1,self.w2)
        
        result= np.argmax(prediction,axis=0)
        final=result.copy()
        
        for i in range(len(result)):
            if result[i]==0:
                final[i]=1
                
            elif result[i]==1:
                final[i]=5
            
            
            elif result[i]==2:
                final[i]=8
        
        
        is_correct=np.equal(final,y)
        
        #return final
        
        return np.sum(is_correct.astype(float)) / y.shape[0]
        
        
        
        
    
    
    
    
    def main(self):
        pass
        
        
        
        
        
        

        
        

    
    
        
    
    
        

In [4]:
if __name__=='__main__':
    #[A] 요구사항 =======================================================
    x_train,x_test,y_train,y_test=load_data() # [A] 요구사항
    #X,y=load_data()
  
    #x_train,x_test,y_train,y_test=shuffle_train_test_split(X,y,shuffle=True)
    #y_train=y_train.astype(np.int)
    #y_test=y_test.astype(np.int)
    
    a=mlp(n_output=3,n_features=x_train.shape[1],n_hidden=100,epochs=500,learning_rate=0.001,minibatches=3)
    
    before_w1=a.w1.copy()
    before_w2=a.w2.copy()
    
    print("학습전 학습용 데이터 예측 결과 : ",a.prediction(x_train,y_train))
    
    
    print("learinig start")
    #[B] 요구사항 =======================================================
    # verbose True 각 epoch당 weight 출력    False 안함
    a.fit(x_train,y_train,verbose=True)
    print("학습 후 train_data 평가: ", a.prediction(x_train,y_train))
    
    
    #[C] 요구사항 =======================================================
    print("학습 후 테스트. 데이터 예측 결과: ",a.prediction(x_test,y_test))

학습전 학습용 데이터 예측 결과 :  0.39322723908216134
learinig start
epoch 0  started
epoch 0   finished
epoch 0 weight_1:  [[ 1.0125228   0.95238849  0.94871958 ... -0.20096793  0.46605674
   0.7657769 ]
 [ 0.64123831 -0.73167395 -0.58999802 ...  0.33910524 -0.77734243
  -0.11530287]
 [-0.81652076  0.72458511  0.75368759 ...  0.00110825  0.62414655
   0.044211  ]
 ...
 [-0.67147091 -0.38509005  0.68948344 ...  0.13548979 -0.72119725
   0.01668765]
 [ 0.41043939  0.81732879 -0.28008435 ... -0.3226506  -0.50462656
  -0.95451927]
 [ 0.59238325  0.65046022  0.99423696 ...  0.13786009  0.50160007
  -0.92373444]]
epoch 0 weight_2:  [[-3.99554298e-01  8.66377167e-01 -4.43391003e-01 -8.37849947e-01
   4.19366522e-01  2.72328829e-01  8.29357477e-01 -8.23496267e-01
  -9.70272778e-01  5.77879752e-01 -5.43584219e-01  2.50888795e-01
   9.09993130e-01  5.85794937e-02 -2.16182704e-01 -1.48578764e-01
  -3.63580614e-01 -3.43798441e-01 -1.97413946e-01  1.26316239e-01
   3.36776317e-01  6.31254942e-01  5.46797909e-0

   4.29013636e-01]]
epoch 2  started
epoch 2   finished
epoch 2 weight_1:  [[ 1.02792729  0.95238849  0.94871958 ... -0.20096793  0.46605674
   0.7657769 ]
 [ 0.6361263  -0.73167395 -0.58999802 ...  0.33910524 -0.77734243
  -0.11530287]
 [-0.8136395   0.72458511  0.75368759 ...  0.00110825  0.62414655
   0.044211  ]
 ...
 [-0.66381936 -0.38509005  0.68948344 ...  0.13548979 -0.72119725
   0.01668765]
 [ 0.41514048  0.81732879 -0.28008435 ... -0.3226506  -0.50462656
  -0.95451927]
 [ 0.58103936  0.65046022  0.99423696 ...  0.13786009  0.50160007
  -0.92373444]]
epoch 2 weight_2:  [[-4.28498314e-01  8.09930814e-01 -3.93840833e-01 -9.13438704e-01
   4.27468039e-01  1.39345504e-01  7.83838830e-01 -7.31872479e-01
  -9.92803476e-01  5.83771387e-01 -5.73164602e-01  2.94517521e-01
   1.00455247e+00  4.03601513e-02 -1.98853986e-01 -1.67447533e-01
  -4.27512797e-01 -3.10981402e-01 -1.43951160e-01  8.56410684e-02
   3.90676897e-01  6.48843571e-01  4.91331382e-01 -9.09140144e-01
  -5.86979346e-01 

epoch 4   finished
epoch 4 weight_1:  [[ 1.03348427  0.95238849  0.94871958 ... -0.20096793  0.46605674
   0.7657769 ]
 [ 0.63719798 -0.73167395 -0.58999802 ...  0.33910524 -0.77734243
  -0.11530287]
 [-0.80934579  0.72458511  0.75368759 ...  0.00110825  0.62414655
   0.044211  ]
 ...
 [-0.66692624 -0.38509005  0.68948344 ...  0.13548979 -0.72119725
   0.01668765]
 [ 0.41746758  0.81732879 -0.28008435 ... -0.3226506  -0.50462656
  -0.95451927]
 [ 0.57189699  0.65046022  0.99423696 ...  0.13786009  0.50160007
  -0.92373444]]
epoch 4 weight_2:  [[-0.44976178  0.78275501 -0.38856327 -0.94641251  0.4332091   0.09014191
   0.76315562 -0.67414903 -1.02854464  0.55654107 -0.57357593  0.31571464
   1.01328681  0.04181031 -0.20777386 -0.183925   -0.43360696 -0.30063919
  -0.10866023  0.06055018  0.37683246  0.64223355  0.4361771  -0.95971358
  -0.5919932  -0.71844986  0.10332618 -0.19521516 -0.15013796 -1.0520336
   0.54338295  0.05124492 -0.94110446  0.67891095 -0.85209996  0.51643572
  -0.775

epoch 6   finished
epoch 6 weight_1:  [[ 1.03656652  0.95238849  0.94871958 ... -0.20096793  0.46605674
   0.7657769 ]
 [ 0.63698012 -0.73167395 -0.58999802 ...  0.33910524 -0.77734243
  -0.11530287]
 [-0.80732686  0.72458511  0.75368759 ...  0.00110825  0.62414655
   0.044211  ]
 ...
 [-0.66833784 -0.38509005  0.68948344 ...  0.13548979 -0.72119725
   0.01668765]
 [ 0.41970558  0.81732879 -0.28008435 ... -0.3226506  -0.50462656
  -0.95451927]
 [ 0.56695191  0.65046022  0.99423696 ...  0.13786009  0.50160007
  -0.92373444]]
epoch 6 weight_2:  [[-0.45408989  0.77625448 -0.38195798 -0.96296817  0.4451126   0.06174559
   0.7569345  -0.62398391 -1.04963068  0.54541205 -0.57577545  0.3308996
   1.02480447  0.04280068 -0.20977627 -0.18618495 -0.43146144 -0.2886148
  -0.07933862  0.05036927  0.36837655  0.64315276  0.40720557 -0.98531026
  -0.58550344 -0.72420041  0.06875732 -0.17626367 -0.17403653 -1.04991413
   0.51059164  0.04911194 -0.97067693  0.65486288 -0.86344266  0.51310637
  -0.7852

epoch 8   finished
epoch 8 weight_1:  [[ 1.03901731  0.95238849  0.94871958 ... -0.20096793  0.46605674
   0.7657769 ]
 [ 0.63666905 -0.73167395 -0.58999802 ...  0.33910524 -0.77734243
  -0.11530287]
 [-0.80602227  0.72458511  0.75368759 ...  0.00110825  0.62414655
   0.044211  ]
 ...
 [-0.67026905 -0.38509005  0.68948344 ...  0.13548979 -0.72119725
   0.01668765]
 [ 0.42295931  0.81732879 -0.28008435 ... -0.3226506  -0.50462656
  -0.95451927]
 [ 0.56348038  0.65046022  0.99423696 ...  0.13786009  0.50160007
  -0.92373444]]
epoch 8 weight_2:  [[-0.45745968  0.77230469 -0.3761032  -0.9736143   0.45359983  0.04194938
   0.75142177 -0.58299315 -1.06587797  0.53954065 -0.581423    0.34007911
   1.03322338  0.0418945  -0.21041601 -0.1883341  -0.43034209 -0.27856108
  -0.05776525  0.04371718  0.36126875  0.64527618  0.38502961 -1.00189045
  -0.57797018 -0.72745544  0.04358083 -0.16038473 -0.19222719 -1.04338039
   0.48586885  0.04676907 -0.99336801  0.63402328 -0.86962007  0.50843476
  -0.79

...

epoch 490   finished
epoch 490 weight_1:  [[ 1.10115534  0.95238849  0.94871958 ... -0.20096793  0.46605674
   0.7657769 ]
 [ 0.59474308 -0.73167395 -0.58999802 ...  0.33910524 -0.77734243
  -0.11530287]
 [-0.70684126  0.72458511  0.75368759 ...  0.00110825  0.62414655
   0.044211  ]
 ...
 [-0.74641598 -0.38509005  0.68948344 ...  0.13548979 -0.72119725
   0.01668765]
 [ 0.49377634  0.81732879 -0.28008435 ... -0.3226506  -0.50462656
  -0.95451927]
 [ 0.52698096  0.65046022  0.99423696 ...  0.13786009  0.50160007
  -0.92373444]]
epoch 490 weight_2:  [[-0.48698368  0.76497217 -0.03493055 -0.90042619  0.11854067  0.14995092
   0.50266432  0.17005974 -1.01090842  0.78125641 -0.76986269  0.4901503
   1.19479507 -0.0965351  -0.18367489 -0.22976485 -0.46887081 -0.10724967
   0.26441931  0.41916121  0.05297663  0.89728327  0.10360118 -0.82906189
  -0.15226581 -0.7567638  -0.0096583   0.12175016 -0.06386485 -0.62350558
   0.0951442  -0.14762149 -1.03797471  0.01842091 -0.3885813   0.6109647
  -

epoch 492   finished
epoch 492 weight_1:  [[ 1.1011523   0.95238849  0.94871958 ... -0.20096793  0.46605674
   0.7657769 ]
 [ 0.59477677 -0.73167395 -0.58999802 ...  0.33910524 -0.77734243
  -0.11530287]
 [-0.7065997   0.72458511  0.75368759 ...  0.00110825  0.62414655
   0.044211  ]
 ...
 [-0.74650209 -0.38509005  0.68948344 ...  0.13548979 -0.72119725
   0.01668765]
 [ 0.49365562  0.81732879 -0.28008435 ... -0.3226506  -0.50462656
  -0.95451927]
 [ 0.52683782  0.65046022  0.99423696 ...  0.13786009  0.50160007
  -0.92373444]]
epoch 492 weight_2:  [[-0.48702539  0.7651084  -0.03425055 -0.9007072   0.1178123   0.15022454
   0.50234608  0.17011393 -1.01064104  0.78140811 -0.77016512  0.49016098
   1.19540384 -0.09672915 -0.18333041 -0.22967452 -0.46873781 -0.10702234
   0.26428794  0.42025708  0.05251534  0.89784562  0.10344439 -0.82935285
  -0.15133541 -0.75729273 -0.00925214  0.12221081 -0.0633492  -0.62313453
   0.09459491 -0.14810877 -1.03789267  0.01740774 -0.38690211  0.61156829
 

epoch 494   finished
epoch 494 weight_1:  [[ 1.10114906  0.95238849  0.94871958 ... -0.20096793  0.46605674
   0.7657769 ]
 [ 0.59481072 -0.73167395 -0.58999802 ...  0.33910524 -0.77734243
  -0.11530287]
 [-0.70635849  0.72458511  0.75368759 ...  0.00110825  0.62414655
   0.044211  ]
 ...
 [-0.74658769 -0.38509005  0.68948344 ...  0.13548979 -0.72119725
   0.01668765]
 [ 0.49353552  0.81732879 -0.28008435 ... -0.3226506  -0.50462656
  -0.95451927]
 [ 0.52669488  0.65046022  0.99423696 ...  0.13786009  0.50160007
  -0.92373444]]
epoch 494 weight_2:  [[-0.48706705  0.76524448 -0.03357293 -0.90099252  0.11708701  0.15049154
   0.50203071  0.17016883 -1.01037524  0.78155909 -0.77047015  0.49016946
   1.19601404 -0.0969216  -0.18298673 -0.22958388 -0.46860512 -0.1067977
   0.26415594  0.42134911  0.05205403  0.89840713  0.10328735 -0.82964415
  -0.15040674 -0.75782527 -0.00884622  0.12266812 -0.06283581 -0.62276503
   0.09404812 -0.14859601 -1.03781466  0.01639989 -0.38522428  0.61217438
  

epoch 497   finished
epoch 497 weight_1:  [[ 1.10114385  0.95238849  0.94871958 ... -0.20096793  0.46605674
   0.7657769 ]
 [ 0.59486211 -0.73167395 -0.58999802 ...  0.33910524 -0.77734243
  -0.11530287]
 [-0.70599729  0.72458511  0.75368759 ...  0.00110825  0.62414655
   0.044211  ]
 ...
 [-0.74671512 -0.38509005  0.68948344 ...  0.13548979 -0.72119725
   0.01668765]
 [ 0.49335656  0.81732879 -0.28008435 ... -0.3226506  -0.50462656
  -0.95451927]
 [ 0.52648085  0.65046022  0.99423696 ...  0.13786009  0.50160007
  -0.92373444]]
epoch 497 weight_2:  [[-0.48712947  0.76544826 -0.03256091 -0.90142857  0.11600463  0.15087974
   0.5015631   0.17025271 -1.00997945  0.78178433 -0.77093242  0.49017798
   1.19693187 -0.09720732 -0.18247284 -0.22944724 -0.46840675 -0.10646576
   0.26395697  0.42297978  0.05136188  0.8992478   0.10305129 -0.83008178
  -0.14901707 -0.75863089 -0.0082377   0.12334782 -0.06206987 -0.62221363
   0.09323273 -0.1493268  -1.03770522  0.01489791 -0.38271     0.61308812
 

epoch 499   finished
epoch 499 weight_1:  [[ 1.10114015  0.95238849  0.94871958 ... -0.20096793  0.46605674
   0.7657769 ]
 [ 0.59489667 -0.73167395 -0.58999802 ...  0.33910524 -0.77734243
  -0.11530287]
 [-0.70575692  0.72458511  0.75368759 ...  0.00110825  0.62414655
   0.044211  ]
 ...
 [-0.74679943 -0.38509005  0.68948344 ...  0.13548979 -0.72119725
   0.01668765]
 [ 0.49323804  0.81732879 -0.28008435 ... -0.3226506  -0.50462656
  -0.95451927]
 [ 0.52633842  0.65046022  0.99423696 ...  0.13786009  0.50160007
  -0.92373444]]
epoch 499 weight_2:  [[-0.48717105  0.76558387 -0.03188915 -0.90172466  0.11528662  0.15113044
   0.50125503  0.17030977 -1.00971752  0.78193375 -0.77124366  0.4901808
   1.19754536 -0.09739587 -0.18213142 -0.22935565 -0.46827504 -0.10624778
   0.2638238   0.42406184  0.05090026  0.89980715  0.10289355 -0.83037399
  -0.14809293 -0.75917253 -0.00783221  0.12379678 -0.06156198 -0.62184794
   0.0926924  -0.14981396 -1.03763728  0.01390299 -0.38103541  0.61370032
  

In [13]:
_,_,_,_,o=a.forward(x_train,a.w1,a.w2)

In [15]:
o.shape

(3, 10808)

In [5]:
print(before_w1)
print('============================================================================================')
print(a.w1)

[[ 0.99568849  0.95238849  0.94871958 ... -0.20096793  0.46605674
   0.7657769 ]
 [ 0.64140065 -0.73167395 -0.58999802 ...  0.33910524 -0.77734243
  -0.11530287]
 [-0.81757242  0.72458511  0.75368759 ...  0.00110825  0.62414655
   0.044211  ]
 ...
 [-0.67991339 -0.38509005  0.68948344 ...  0.13548979 -0.72119725
   0.01668765]
 [ 0.40763172  0.81732879 -0.28008435 ... -0.3226506  -0.50462656
  -0.95451927]
 [ 0.60234914  0.65046022  0.99423696 ...  0.13786009  0.50160007
  -0.92373444]]
[[ 1.10114015  0.95238849  0.94871958 ... -0.20096793  0.46605674
   0.7657769 ]
 [ 0.59489667 -0.73167395 -0.58999802 ...  0.33910524 -0.77734243
  -0.11530287]
 [-0.70575692  0.72458511  0.75368759 ...  0.00110825  0.62414655
   0.044211  ]
 ...
 [-0.74679943 -0.38509005  0.68948344 ...  0.13548979 -0.72119725
   0.01668765]
 [ 0.49323804  0.81732879 -0.28008435 ... -0.3226506  -0.50462656
  -0.95451927]
 [ 0.52633842  0.65046022  0.99423696 ...  0.13786009  0.50160007
  -0.92373444]]
